In [1]:
import json
import re

In [18]:
with open('real_dict_2000_new_only_delete_question_noti_new_and_space.json','r') as dict_file:
    real_dict = json.load(dict_file)
    print(real_dict.keys())
    list_name_activity = real_dict['time']
    for name_activity in list_name_activity:
        if "hội" in name_activity:
            print(name_activity)
        

dict_keys(['name_activity', 'type_activity', 'holder', 'time', 'name_place', 'address', 'reward', 'contact', 'register', 'works', 'joiner'])


In [16]:
with open('temp_data/real_db_915.json','r') as db_file:
    list_data = json.load(db_file)
#     print(list_data)
    res = list_data
    for data in list_data:
        check_not_match_name_activity = False
#         if 'name_activity' not in list(data.keys()):
#             break
        for name_activity in data['name_activity']:
            if name_activity not in list_name_activity:
                check_not_match_name_activity = True
                break
        if check_not_match_name_activity == True:
            res.remove(data)
    print(len(res))
    with open('temp_data/real_db_769.json', 'w+') as new_db_file:
        json.dump(res,new_db_file,ensure_ascii=False)
    
                
            
        

769


In [20]:
with open('list_constants.json','r') as constants_file:
    list_constants = json.load(constants_file)
    list_pattern_time = list_constants['list_pattern_time']
    for pattern in list_pattern_time:
        if re.findall(pattern,'305') != []:
            print(pattern)

\d\d\d


In [11]:
from temp_agent_action_gen import *
from message_handler import *
from agen_response_gen import *



In [13]:
def find_all_entity(intent,input_sentence):
    print("duongcc")
    normalized_input_sentence = compound2unicode(input_sentence)
    normalized_input_sentence = delete_extra_word(normalized_input_sentence,list_extra_word)
    
    result_entity_dict={}
    list_order_entity_name=map_intent_to_list_order_entity_name[intent]
    print(normalized_input_sentence)
    if 'time' in list_order_entity_name:
        for pattern_time in list_pattern_time:
            if re.findall(pattern_time,normalized_input_sentence)!=[]:
                # print("pattern_time :{0}".format(pattern_time))
                if 'time' not in result_entity_dict:
                    result_entity_dict['time'] = delete_last_space_list(re.findall(pattern_time,normalized_input_sentence))
                else:
                    result_entity_dict['time'].extend(delete_last_space_list(re.findall(pattern_time,normalized_input_sentence)))
                
                normalized_input_sentence = re.sub(pattern_time,' '.join(['✪']*(pattern_time.count(' ')+1)),normalized_input_sentence)
        # if 'time' in result_entity_dict:
        #     print(result_entity_dict['time'])
    if 'reward' in list_order_entity_name:
        for pattern_reward in list_pattern_reward:
            if re.findall(pattern_reward,normalized_input_sentence)!=[]:
                print("pattern_reward :{0}".format(pattern_reward))
                if 'reward' not in result_entity_dict:
                    result_entity_dict['reward'] = delete_last_space_list(re.findall(pattern_reward,normalized_input_sentence))
                else:
                    result_entity_dict['reward'].extend(delete_last_space_list(re.findall(pattern_reward,normalized_input_sentence)))
                
                normalized_input_sentence = re.sub(pattern_reward,' '.join(['✪']*(pattern_reward.count(' ')+1)),normalized_input_sentence)
        # if 'reward' in result_entity_dict:
        #     print(result_entity_dict['reward'])
    matching_threshold = 0.0
    longest_common_length, end_common_index = None, None
    
    map_entity_name_to_threshold={}
    for entity_name in list_order_entity_name:
        if entity_name in ['time','address']:
            map_entity_name_to_threshold[entity_name]=1
        elif entity_name in ['name_activity','contact','joiner','holder','type_activity','name_place']:
            map_entity_name_to_threshold[entity_name]=2
        elif entity_name in ['works','reward']:
            map_entity_name_to_threshold[entity_name]=2
        elif entity_name in ['register']:
            map_entity_name_to_threshold[entity_name]=3


    ordered_real_dict = OrderedDict()
    for entity_name in map_intent_to_list_order_entity_name[intent]:
        ordered_real_dict[entity_name] = real_dict[entity_name]
    for entity_name, list_entity in ordered_real_dict.items():
        # print(entity_name)
        list_entity = [entity.lower() for entity in list_entity]
        # print("input sentence: {0}".format(normalized_input_sentence))
        if entity_name in ["works","register","reward"]:
            matching_threshold = 0.15
        elif entity_name == "joiner":
            matching_threshold = 0.2
        else:
            matching_threshold = 0.5
        print("000. sentence:{0}".format(normalized_input_sentence))
        catch_entity_threshold_loop = 0
        while True:
            if catch_entity_threshold_loop > 5:
                break
            list_dict_longest_common_entity = find_entity_longest_common(normalized_input_sentence,list_entity,entity_name)
#             print(list_dict_longest_common_entity)
                #     [{'longest_common_entity_index': 0,
                #   'longest_common_length': 3,
                #   'end_common_index': 9}]
            

            ##find the most match longest common match (calculate by length of token match in sentence 
                                                                #/ length of entity )
                            ##{'greatest_match_entity_index':0,'longest_common_length':3,'end_common_index':9}
            if list_dict_longest_common_entity == []:
                break
            if list_dict_longest_common_entity[0]['longest_common_length'] < map_entity_name_to_threshold[entity_name] :
                break
            
            list_sentence_token = normalized_input_sentence.split(' ')
#             print("list_sentence_token :{0}".format(list_sentence_token))
            greatest_entity_index=None
            greatest_common_length = None
            greatest_end_common_index = None
            max_match_entity = 0.0
#             print("common entity :{0}".format(list_dict_longest_common_entity))
            for dict_longest_common_entity in list_dict_longest_common_entity:
#                 print("0. dict_longest_common_entity: {0}".format(dict_longest_common_entity))

#                     print("duong")
#                 print("0.1 entity: {0}".format(list_entity[dict_longest_common_entity['longest_common_entity_index']]))
                longest_common_entity_index = dict_longest_common_entity['longest_common_entity_index']
                longest_common_length = dict_longest_common_entity['longest_common_length']
                end_common_index = dict_longest_common_entity['end_common_index']
                
                list_sentence_token_match = list_sentence_token[end_common_index - longest_common_length+1:end_common_index+1]
                if entity_name == "type_activity":
                    if "ban chỉ huy" in normalized_input_sentence or "ban tổ chức" in normalized_input_sentence or "bch" in normalized_input_sentence or "btc" in normalized_input_sentence:
                        continue
                    #nếu chỉ là các câu inform 1 entity mà câu đó không phải là câu inform tên 1 hoạt động thì không cần xét
                    if "inform" not in intent or "name_activity" in intent:
                        list_name_activity = ordered_real_dict['name_activity']
                        check_in_name = False
                        for name_activity in list_name_activity:
                            #nếu loại hoạt động nằm trọn trong bất kì 1 tên hoạt động 
                            # thì không lấy
                            if  name_activity.find(' '.join(list_sentence_token_match)) > 0:
                                check_in_name = True
                                break
                        if check_in_name == True:
                            continue
                
                if entity_name == "holder":
                    # nếu holder mà trước đó có từ chỉ nơi chốn : ở, tại => không là holder mà là  
                    # name_place
                    if end_common_index - longest_common_length >= 0:
                        if list_sentence_token[end_common_index - longest_common_length] in ["ở","tại","trước","sau","trong"]:
                            if 'name_place' in result_entity_dict:
            #                     result_entity_dict[entity_name].append(list_entity[greatest_entity_index])
                                result_entity_dict['name_place'].append(' '.join(list_sentence_token_match))
                            else:
            #                     result_entity_dict[entity_name] = [list_entity[greatest_entity_index]]
                                result_entity_dict['name_place'] = [' '.join(list_sentence_token_match)]
                            list_sentence_token[end_common_index - longest_common_length +1 :end_common_index +1] = ["✪"]*longest_common_length
                            normalized_input_sentence = ' '.join(list_sentence_token)
                            continue

#                 print("2. list_sentence_token_match : {0}".format(list_sentence_token_match))
                list_temp_longest_entity_token = list_entity[longest_common_entity_index].split(' ')
#                 print("3. list_temp_longest_entity_token : {0}".format(list_temp_longest_entity_token))
                if entity_name in ["works","register","reward"]:
#                     print("list_temp_longest_entity_token :{0}".format(list_temp_longest_entity_token))
#                     print("list_sentence_token_match :{0}".format(list_sentence_token_match))
                    _,longest_common_length_entity,end_common_index_entity = lcs_length_ta(list_temp_longest_entity_token,list_sentence_token_match)
                    list_entity_token_match = list_temp_longest_entity_token[end_common_index_entity - longest_common_length_entity +1 :end_common_index_entity +1]
                    score = len(list_entity_token_match)/float(len(list_temp_longest_entity_token))
#                     print("list_entity_token_match: {0}".format(list_entity_token_match))
#                     print("list_temp_longest_entity_token:{0}".format(list_temp_longest_entity_token))
#                     print("score :{0}".format(score))
                    
                else:
                    score = len(list_sentence_token_match)/float(len(list_temp_longest_entity_token))
                if score > max_match_entity:
#                     max_match_entity = len(list_sentence_token_match)/float(len(list_temp_longest_entity_token))
                    max_match_entity = score
                    greatest_entity_index = longest_common_entity_index
                    greatest_common_length = longest_common_length
                    greatest_end_common_index = end_common_index
#             print(list_sentence_token)
#             print(greatest_common_length)
#             print(greatest_end_common_index)
#             print("longest_common_length: {0}".format(longest_common_length))
#             print("end_common_index: {0}".format(end_common_index))
#             print("1. greatest_common_length : {0}".format(greatest_common_length))
            # print(max_match_entity)
            # print("2. greatest entity : {0}".format(list_entity[greatest_entity_index]))
#             print("2.1 greatest_end_common_index: {0}".format(greatest_end_common_index))
#             print("3. sentence match: {0}".format(list_sentence_token[greatest_end_common_index - greatest_common_length +1 :greatest_end_common_index +1]))
            if greatest_common_length != None:
                if greatest_common_length >= map_entity_name_to_threshold[entity_name] and max_match_entity > matching_threshold:
                    # if entity_name in ['name_activity','type_activity']:
                    #     result = list_entity[greatest_entity_index]
                    # else:
                    #     result = ' '.join(list_sentence_token[greatest_end_common_index - greatest_common_length +1 :greatest_end_common_index +1])
                    
                    result = ' '.join(list_sentence_token[greatest_end_common_index - greatest_common_length +1 :greatest_end_common_index +1])
                    if entity_name in result_entity_dict:
    #                     result_entity_dict[entity_name].append(list_entity[greatest_entity_index])
                        result_entity_dict[entity_name].append(result)
                    else:
    #                     result_entity_dict[entity_name] = [list_entity[greatest_entity_index]]
                        result_entity_dict[entity_name] = [result]
    #                 list_sentence_token = list_sentence_token[:greatest_end_common_index - greatest_common_length + 1] + list_sentence_token[greatest_end_common_index +1 :]
                    list_sentence_token[greatest_end_common_index - greatest_common_length +1 :greatest_end_common_index +1] = ["✪"]*greatest_common_length
                    normalized_input_sentence = ' '.join(list_sentence_token)
            catch_entity_threshold_loop = catch_entity_threshold_loop + 1
            # print("output sentence: {0}".format(normalized_input_sentence))
    return result_entity_dict

In [14]:


CONSTANT_FILE_PATH = 'constants.json'
with open(CONSTANT_FILE_PATH) as f:
    constants = json.load(f)

file_path_dict = constants['db_file_paths']
DATABASE_FILE_PATH = file_path_dict['database']

database= json.load(open(DATABASE_FILE_PATH,encoding='utf-8'))
state_tracker = StateTracker(database, constants)
dqn_agent = DQNAgent(state_tracker.get_state_size(), constants)
#TEST
if __name__ == '__main__':
    print(process_message_to_user_request("khi nào thì diễn ra hiến máu tình nguyện đợt ii ở hội trường lớn đh ngân hàng vậy",state_tracker))
    

✪ ✪ ✪ diễn ra hiến máu tình nguyện đợt ii ở hội trường lớn đh ngân hàng vậy
{'intent': 'request', 'inform_slots': {'time': ['hội trường'], 'name_activity': ['hiến máu tình nguyện đợt ii'], 'holder': ['đh ngân hàng']}, 'request_slots': {'time': 'UNK'}}


In [19]:
with open('list_constants.json','r') as constant_file:
    constant = json.load(constant_file)
    list_pattern_time = constant['list_pattern_time']
    res = []
    for pattern in list_pattern_time:
        if "chủ nhật" not in pattern:
            res.append(pattern)
            
    constant['list_pattern_time'] = res
    with open('list_constants_new.json', 'w+') as new_constant_file:
        json.dump(constant,new_constant_file,ensure_ascii=False)
    
    

\d\d\d?h \d\d\d?h các buổi tối trong tuần \( từ thứ \d đến chủ nhật \) , bắt đầu từ thứ \d tuần này \( \d\d?[/-]\d\d \)
\d\d\d?h\d?\d?’ ngày \d - \d - \d\d\d\d \( chủ nhật \) \d\d?h\d?\d?’ ngày \d\d - \d - \d\d\d\d \( thứ \d \)
từ \d\d?h đến \d\d\d?h các ngày chủ nhật . \( \d\d?[/-]\d\d ; \d[/-]\d\d ; \d\d?[/-]\d\d ; \d\d?[/-]\d\d \)
chủ nhật , ngày \d\d?[/-]\d\d?[/-]\d\d\d?\d? , từ \d\d?h\d?\d? sáng , dự kiến kết thúc lúc \d\d\d?h\d?\d?
\d\d?h\d?\d? - \d\d\d?h\d?\d? chủ nhật ngày \d\d?[/-]\d\d?[/-]\d\d\d?\d? và \d\d?[/-]\d\d?[/-]\d\d\d?\d?
từ ngày \d\d?[/-]\d\d đến \d\d?[/-] \d\d?[/-] \d\d\d\d \( vào các ngày thứ \d , chủ nhật hàng tuần \)
từ sáng thứ \d , \d\d?[/-]\d\d?[/-]\d\d\d?\d? đến trưa chủ nhật , \d\d?[/-]\d\d?[/-]\d\d\d?\d?
từ \d\d?h ngày \d\d?[/-]\d \d\d\d?h ngày \d[/-]\d\d?[/-]\d\d\d?\d? . \( thứ \d & chủ nhật \)
thứ \d & chủ nhật hằng tuần từ \d\d?[/-]\d\d?[/-]\d\d\d?\d? \d\d?[/-]\d\d?[/-]\d\d\d?\d?
từ \d\d\d?h\d?\d? đến \d\d\d?h\d?\d? ngày \d\d?[/-]\d\d?[/-]\d\d\d?\d? \( 